In [14]:
import logging
logging.basicConfig(
    style='{',
    format='{asctime} [{filename}:{lineno} in {funcName}] {levelname} - {message}',
    handlers=[
        logging.FileHandler(".log", 'w'),
        logging.StreamHandler()
    ],
    level=logging.INFO
)
logging.info('Importing...')
import os
import numpy as np
import torch
from torch.nn import Module, ModuleList
from transformers import PreTrainedModel
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
from accelerate import init_empty_weights
from accelerate.utils import find_tied_parameters, named_module_tensors, set_module_tensor_to_device

from policy import Policy
logging.info('Done!')

2023-09-24 11:25:08,190 [361951628.py:11 in <module>] INFO - Importing...
2023-09-24 11:25:08,192 [361951628.py:22 in <module>] INFO - Done!


In [15]:
checkpoint = "facebook/opt-125m" 
# checkpoint = "facebook/opt-13b" # 1.3b 6.7b 13b 30b 66b 
offload_folder = 'offload/' + checkpoint.replace('/', '.')

# empty model
config = AutoConfig.from_pretrained(checkpoint)
with init_empty_weights():
    model = AutoModelForCausalLM.from_config(config)
model.tie_weights()

In [16]:
class AttrDict(dict):
    __slots__ = () 
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__


policy = Policy(
    gpu_batch_size=16, 
    num_gpu_batches=8, 
    weights_gpu_percent=0.0, 
    weights_cpu_percent=0.3, 
    cache_gpu_percent=0.0, 
    cache_cpu_percent=0.2, 
    act_gpu_percent=0.0, 
    act_cpu_percent=0.5, 
    overlap=True, 
    pin_weight=True,
)

def get_layers_dict(lm_model: Module, prefix: str='') -> dict:
    # return a dict of {layer_name : layer_module ('meta')} with only leaf nodes & transformer layers
    layers_dict = {}
    for name, module in lm_model.named_children():
        # leaf nodes
        if len(list(module.named_children())) == 0:
            layers_dict[prefix+name] = module
        # ModuleList: transformer  
        elif isinstance(module, ModuleList):
            for block_name, block_module in module.named_children():
                layers_dict[prefix+name+'.'+block_name] = block_module
        else:
            layers_dict.update(get_layers_dict(module, prefix+name+'.'))
    return layers_dict

def get_device(cur_percent, percents, choices):
    # choose a device (gpu / cpu / disk) for a weight tensor by its percent of size
    percents = np.cumsum(percents)
    assert np.abs(percents[-1] - 1.0) < 1e-5, f'{percents}'

    for i in range(len(percents)):
        if cur_percent < percents[i]:
            return choices[i]
    return choices[-1]

def get_policy_weight_map(model: PreTrainedModel, policy: Policy):
    """{module_name: device}"""
    assert model.device == torch.device('meta'), 'model is not on device meta.'
    
    # to ensure the tied params are allocated to the same device in the weight_map
    model.tie_weights()
    tied_params = find_tied_parameters(model)

    # layers to be scheduled
    layers_dict = get_layers_dict(model)

    # device assignment for each tensor in the model
    weight_assign_dict = {}
    devices = ['cuda', 'cpu', 'disk']
    percents_target = np.array([
        policy.weights_gpu_percent, 
        policy.weights_cpu_percent, 
        policy.weights_disk_percent
    ])
    
    # model size (parameters + buffers), here we do not repeatly sum the tied paramters 
    size_total = sum(np.prod(tensor.shape) for _, tensor in named_module_tensors(model, include_buffers=True, recurse=True))
    size_done, size_todo = 0, size_total
    percents_done, percents_todo = 0 * percents_target, percents_target  

    for layer_name, layer_module in layers_dict.items():
        # current layer
        tensor_sizes = [np.prod(tensor.shape) for _, tensor in named_module_tensors(layer_module, include_buffers=True, recurse=True)]
        tensor_sizes_cumsum = np.cumsum(tensor_sizes)

        device_allo_size_dict = {device: 0 for device in devices} # to balance the percents
        for i, (tensor_name, tensor) in enumerate(named_module_tensors(layer_module, include_buffers=True, recurse=True)):
            abs_tensor_name = layer_name + '.' + tensor_name

            def find_processed_tied(abs_tensor_name, tied_params, weight_assign_dict):
                # find the processed parameter (in weight_assign_dict) of the tied parameters.
                for tp in tied_params:
                    if abs_tensor_name in tp:
                        for p in tp:
                            if p in weight_assign_dict:
                                return p, tuple(tp)
                return None
            
            processed_tied = find_processed_tied(abs_tensor_name, tied_params, weight_assign_dict) 
            if processed_tied: # this tensor is tied and processed.
                p, tp = processed_tied
                weight_assign_dict[abs_tensor_name] = {
                    # 'shape':  tensor.shape,
                    'assigned_device': weight_assign_dict[p]['assigned_device'],
                    'tied': tp
                }
            else:
                mid_percent = (tensor_sizes_cumsum[i] - tensor_sizes[i] / 2) / tensor_sizes_cumsum[-1] # tensor mid size percent 
                device = get_device(mid_percent, percents_todo, devices)
                weight_assign_dict[abs_tensor_name] = {
                    'shape':  tensor.shape,
                    'assigned_device': device
                }
                
                device_allo_size_dict[device] += tensor_sizes[i]

        # update percents_todo
        size_layer = sum(device_allo_size_dict.values())
        if size_layer > 0:
            device_allo_percents = np.array([device_allo_size_dict[device] * 1. for device in devices]) / size_layer
            percents_done = (percents_done * size_done + device_allo_percents * size_layer) / (size_done + size_layer)      
        size_done += size_layer
        size_todo -= size_layer
        if size_todo > 0:
            percents_todo = (size_total * percents_target - size_done * percents_done) / size_todo 
        
        logging.info(f'{layer_name}, {percents_done}, size_todo: {size_todo}')


    device_map = {k:v['assigned_device'] for k, v in weight_assign_dict.items()}
    logging.info('device_map is prepared!')

    mem_g = sum([np.prod(v['shape']) for _, v in weight_assign_dict.items() if 'cuda' in v['assigned_device'] and 'shape' in v]) * 2 / (2 ** 30)
    mem_c = sum([np.prod(v['shape']) for _, v in weight_assign_dict.items() if v['assigned_device'] == 'cpu' and 'shape' in v]) * 2 / (2 ** 30)
    mem_d = sum([np.prod(v['shape']) for _, v in weight_assign_dict.items() if v['assigned_device'] == 'disk' and 'shape' in v]) * 2 / (2 ** 30)
    mem = mem_d + mem_c + mem_g
    logging.info(f'CausalLM {checkpoint} is to be loaded on: ' 
                 f'\nGPU Mem {mem_g:.2f} GiB ({mem_g / mem:.2%}), ' 
                 f'CPU Mem {mem_c:.2f} GiB ({mem_c / mem:.2%}), '
                 f'Disk Mem {mem_d:.2f} Gib ({mem_d / mem:.2%})')
    
    # prepare output
    output = {
        'model': model,
        'tied_params': tied_params,
        'layers_dict': layers_dict,
        'weight_assign_dict': weight_assign_dict,
        'device_map': device_map
    }
    output = AttrDict(output)
    return output

output = get_policy_weight_map(model, policy)
policy_device_map = output.device_map
flexgen_layers = output.layers_dict

2023-09-24 11:25:08,367 [4043888472.py:116 in get_policy_weight_map] INFO - model.decoder.embed_tokens, [0. 0. 1.], size_todo: 86630400
2023-09-24 11:25:08,369 [4043888472.py:116 in get_policy_weight_map] INFO - model.decoder.embed_positions, [0. 0. 1.], size_todo: 85056000
2023-09-24 11:25:08,370 [4043888472.py:116 in get_policy_weight_map] INFO - model.decoder.final_layer_norm, [0.00000000e+00 1.91116887e-05 9.99980888e-01], size_todo: 85054464
2023-09-24 11:25:08,372 [4043888472.py:116 in get_policy_weight_map] INFO - model.decoder.layers.0, [0.         0.05002193 0.94997807], size_todo: 77966592
2023-09-24 11:25:08,373 [4043888472.py:116 in get_policy_weight_map] INFO - model.decoder.layers.1, [0.         0.08698539 0.91301461], size_todo: 70878720
2023-09-24 11:25:08,375 [4043888472.py:116 in get_policy_weight_map] INFO - model.decoder.layers.2, [0.         0.11542163 0.88457837], size_todo: 63790848
2023-09-24 11:25:08,377 [4043888472.py:116 in get_policy_weight_map] INFO - model

In [17]:
def check_disk(checkpoint, offload_folder):
    if not os.path.isdir(offload_folder):
        return False 
    
    config = AutoConfig.from_pretrained(checkpoint)
    with init_empty_weights():
        model = AutoModelForCausalLM.from_config(config)
    model.tie_weights()
    tensor_names = [n for n, _ in named_module_tensors(model, include_buffers=True, recurse=True)]
    dat_file_names = [file[:-4] for file in os.listdir(offload_folder) if file.endswith('.dat')]
    # logging.info(set(tensor_names) - set(dat_file_names), set(dat_file_names) - set(tensor_names))
    return len(set(tensor_names) - set(dat_file_names)) == 0

if not check_disk(checkpoint, offload_folder):
    # download and process to .dat files
    disk_weight_map = {name:'disk' for name in policy_device_map}
    try:
        AutoModelForCausalLM.from_pretrained(
            checkpoint, 
            device_map=disk_weight_map, 
            offload_folder=offload_folder, 
            offload_state_dict=True
        )
    except:
        pass

if check_disk(checkpoint, offload_folder):
    logging.info(f'The whole model has been downloaded an processed to offload_folder: \'{offload_folder}\'')
else:
    err_msg = 'Mismatch between offload folder and model'
    logging.error(err_msg)
    raise RuntimeError(err_msg)

# get empty model
config = AutoConfig.from_pretrained(checkpoint)
with init_empty_weights():
    model = AutoModelForCausalLM.from_config(config)
model.tie_weights()
model.eval()
logging.info(f'Got empty CausalLM: \'{checkpoint}\' on meta device.')

tied_params = find_tied_parameters(model)
print(tied_params)

2023-09-24 11:25:08,656 [1111749824.py:28 in <module>] INFO - The whole model has been downloaded an processed to offload_folder: 'offload/facebook.opt-125m'
2023-09-24 11:25:08,773 [1111749824.py:40 in <module>] INFO - Got empty CausalLM: 'facebook/opt-125m' on meta device.


[['lm_head.weight', 'model.decoder.embed_tokens.weight']]


In [18]:
def get_obj_from_name(lm_model, name):
    splits = name.split('.')
    module = lm_model
    for split in splits:
        if split == '': 
            continue 

        new_module = getattr(module, split)
        if new_module is None:
            raise ValueError(f"{module} has no attribute {split}.")
        module = new_module
    return module 

In [19]:
import json
from tqdm import tqdm 
import gc 

dat_files = [f for f in os.listdir(offload_folder) if f.endswith('.dat')]
with open(os.path.join(offload_folder, 'index.json'), 'r') as f:
    index = json.load(f) # {name: {dtype, shape}}

def get_tied_target(tensor_name):
    # if tensor_name is tied and without a .dat file, if it is not tied, return itself
    for group in tied_params:
        if tensor_name in group:
            for name in group:
                if name + '.dat' in dat_files:
                    return name 
    return tensor_name

def flexgen_load_module_tensor(model, tensor_name, device):
    tensor = get_obj_from_name(model, tensor_name)
    if tensor.device == device:
        return 
    
    # else
    old_tensor_name = tensor_name
    
    tensor_name = get_tied_target(tensor_name) 
    metadata = index[tensor_name]

    # copied from accelerate.utils.offload
    shape = tuple(metadata["shape"])
    if shape == ():
        # NumPy memory-mapped arrays can't have 0 dims so it was saved as 1d tensor
        shape = (1,)

    dtype = metadata["dtype"]
    if dtype == "bfloat16":
        # NumPy does not support bfloat16 so this was saved as a int16
        dtype = "int16"
    
    # load .dat file
    save_path = os.path.join(offload_folder, tensor_name + '.dat')

    # to device 
    np_memmap = np.memmap(save_path, dtype=dtype, shape=shape, mode='r') 
    tmp = torch.from_numpy(np_memmap).to(device) 
    set_module_tensor_to_device(model, old_tensor_name, device, tmp)

def flexgen_offload_module_tensor(model, tensor_name):
    tensor = get_obj_from_name(model, tensor_name)
    device = policy_device_map[tensor_name]
    device = device if device != 'disk' else 'meta' 
    if tensor.device != device:
        set_module_tensor_to_device(model, tensor_name, device, tensor) # gtoc, ctog

def policy_init(model, policy_device_map):
    for tensor_name, device in tqdm(policy_device_map.items(), desc='model init: loading by policy...'):
        if device != 'disk':
            flexgen_load_module_tensor(model, tensor_name, device) 

    logging.info('model has been loaded by policy.')        

policy_init(model, policy_device_map)

model init: loading by policy...: 100%|██████████| 197/197 [00:00<00:00, 4876.87it/s]
2023-09-24 11:25:08,847 [3593259108.py:60 in policy_init] INFO - model has been loaded by policy.


In [35]:
def get_info(obj): # recursive

    if isinstance(obj, tuple):
        return tuple(get_info(o) for o in obj)
    elif isinstance(obj, dict):
        return {k:get_info(v) for k, v in obj.items()}
    elif isinstance(obj, torch.Tensor):
        return obj.size()
    else:
        return type(obj)
              


In [41]:
from accelerate.hooks import (
    ModelHook, 
    SequentialHook, 
    add_hook_to_module, 
    remove_hook_from_module
)
# TODO: add_minibatch_hook, remove_minibatch_hook

from accelerate.utils import (
    find_device,
    named_module_tensors,
    send_to_device,
    set_module_tensor_to_device,
)

# global buffers: {layer_name: value_holder}
# weight_home = {}
# weight_load_buf = {}

act_home = {}
act_load_buf = {}
act_store_buf = {}

kv_home = {} 
kv_load_buf = {}
kv_store_buf = {}

# TODO: cuda streams / cpu threads (?)


from typing import Optional, Union, Mapping
class LayerWeightHook(ModelHook):
    def __init__(
        self,
        model, 
        layer_name,
        next_layer_name,
        compute_device,
    ):
        self.model = model
        self.layer_name = layer_name
        self.next_layer_name = next_layer_name
        self.compute_device = compute_device

        # get weight names
        layer_module = get_obj_from_name(model, layer_name)
        self.weight_names = [layer_name + '.' + name for name, _ in named_module_tensors(layer_module, True, True)]
        dat_files = [os.path.join(offload_folder, get_tied_target(w) + '.dat') for w in self.weight_names]
        assert all([self.check_dat(f) for f in dat_files]), f'dat file error, {dat_files}'
        
        if next_layer_name:
            next_layer_module = get_obj_from_name(model, next_layer_name)
            self.next_weight_names = [next_layer_name + '.' + name for name, _ in named_module_tensors(next_layer_module, True, True)]
            dat_files = [os.path.join(offload_folder, get_tied_target(w) + '.dat') for w in self.next_weight_names]
            assert all([self.check_dat(f) for f in dat_files]), f'dat file error, {dat_files}'

        
    def check_dat(self, dat_file):
        return os.path.isfile(dat_file)

    def init_hook(self, module):
        return module 

    def load_layer(self, weight_names):
        for w in weight_names:
            flexgen_load_module_tensor(self.model, w, self.compute_device)

    def offload_layer(self, weight_names):
        for w in weight_names:
            flexgen_offload_module_tensor(self.model, w)
    
    def pre_forward(self, module: Module, *args, **kwargs):
        logging.info(f'pre {self.layer_name} forward,'
                     f'\n args:{get_info(args)},'
                     f'\n kwargs: {get_info(kwargs)}')


        self.load_layer(self.weight_names) 
        if self.next_layer_name:
            self.load_layer(self.next_weight_names) 

        return args, kwargs
    
    def post_forward(self, module, output):
        
        logging.info(f'post {self.layer_name} forward, \noutput: {get_info(output)}\n\n')

        self.offload_layer(self.weight_names)
        return output
    
    def detach_hook(self, module):
        return module 

import functools

def add_minibatch_hook(module, num_gpu_batches):
    if hasattr(module, "_hf_hook") and hasattr(module, "_old_forward"):
        # If we already put some hook on this module, we replace it with the new one.
        old_forward = module._old_forward
    else:
        old_forward = module.forward
        module._old_forward = old_forward

    @functools.wraps(old_forward)
    def new_forward(*args, **kwargs):
        args, kwargs = module._hf_hook.pre_forward(module, *args, **kwargs)
        if module._hf_hook.no_grad:
            with torch.no_grad():
                args_list = []
                kwargs_list = []
                for k in range(num_gpu_batches):
                    # args_list 
                    args_k = []
                    for arg in args:
                        if isinstance(arg, torch.Tensor):
                            assert arg.size(0) % num_gpu_batches == 0
                            batch_size = arg.size(0) // num_gpu_batches
                            arg_k = arg[k * batch_size: (k + 1) * batch_size]
                        else: # int
                            arg_k = arg 
                        args_k.append(arg_k) 

                    args_list.append(args_k) 

                    # kwargs_list
                    kwargs_k = {}
                    for key, value in kwargs.items():
                        if isinstance(value, torch.Tensor):
                            assert value.size(0) % num_gpu_batches == 0
                            batch_size = arg.size(0) // num_gpu_batches
                            kwargs_k[key] = value[k * batch_size: (k + 1) * batch_size]
                        elif isinstance(value, tuple) and isinstance(value[0], torch.Tensor): # kv cache 
                            assert value[0].size(0) % num_gpu_batches == 0
                            batch_size = arg.size(0) // num_gpu_batches
                            kwargs_k[key] = tuple(v[k * batch_size: (k + 1) * batch_size] for v in value)
                        else:
                            kwargs_k[key] = value 
                    kwargs_list.append(kwargs_k)


                output_list = []
                for a, kw in zip(args_list, kwargs_list):
                    output = old_forward(*a, **kw)
                    output_list.append(output) 
                
                output = output_list 
                # output = old_forward(*args, **kwargs)
        else:
            output = old_forward(*args, **kwargs)
        return module._hf_hook.post_forward(module, output)

    module.forward = new_forward
    return module  

def remove_minibatch_hook(module, recurse):
    if hasattr(module, "_hf_hook"):
        module._hf_hook.detach_hook(module)
        delattr(module, "_hf_hook")

    if hasattr(module, "_old_forward"):
        module.forward = module._old_forward
        delattr(module, "_old_forward")

    if recurse:
        for child in module.children():
            remove_hook_from_module(child, recurse)

    return module

class LayerActHook(ModelHook): pass 
class LayerKVCacheHook(ModelHook): pass 

def clear_hooks(model):
    remove_hook_from_module(model, recurse=True)


clear_hooks(model)

compute_device = 'cpu' 

layer_names = list(flexgen_layers)
for i, layer_name in enumerate(layer_names): # layer names
    layer_module = get_obj_from_name(model, layer_name)
    next_layer_name = layer_names[i + 1] if i < len(layer_names) - 1 else None 

    layer_weight_hook = LayerWeightHook(
        model=model, layer_name=layer_name, next_layer_name=next_layer_name, compute_device=compute_device)
    add_hook_to_module(layer_module, layer_weight_hook, append=True)
    # break


In [42]:

# generate test

prompts = [
    'Who are you? Are you conscious?',
    'Where is Deutschland?',
    'How is Huawei Mate 60 Pro?'
] * 6

prompt_len = 10

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
inputs = tokenizer(prompts, padding="max_length", max_length=prompt_len, return_tensors="pt")

# Generate
generate_ids = model.generate(
    inputs.input_ids, 
    max_length=30 + prompt_len,
    # num_beams=2,
    # num_beam_groups=2,
    # diversity_penalty=0.1,
    do_sample=True,
)

output_texts = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
for output_text in output_texts:
    logging.info(output_text)
    logging.info('-' * 10)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/home/fsuser/miniconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1535: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on meta. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('meta') before running `.generate()`.
  warnings.warn(
2023-09-24 11:56:48,201 [1792503362.py:73 in pre_forward] INFO - pre model.decoder.embed_tokens forward,
 args:(torch.Size([18, 10]),),
 kwargs: {}
2023-09-24 11:56:48,207 [1792503362.py:86 in post_forward] INFO - post model.decoder.embed_tokens forward, 
output: torch.Size([18, 10, 768])


2023-09-24 11:56:48,210 [1792503362

2023-09-24 11:56:48,349 [1792503362.py:86 in post_forward] INFO - post model.decoder.layers.0 forward, 
output: (torch.Size([18, 10, 768]), (torch.Size([18, 12, 10, 64]), torch.Size([18, 12, 10, 64])))


2023-09-24 11:56:48,356 [1792503362.py:73 in pre_forward] INFO - pre model.decoder.layers.1 forward,
 args:(torch.Size([18, 10, 768]),),
 kwargs: {'attention_mask': torch.Size([18, 1, 10, 10]), 'layer_head_mask': <class 'NoneType'>, 'past_key_value': <class 'NoneType'>, 'output_attentions': <class 'bool'>, 'use_cache': <class 'bool'>}
2023-09-24 11:56:48,465 [1792503362.py:86 in post_forward] INFO - post model.decoder.layers.1 forward, 
output: (torch.Size([18, 10, 768]), (torch.Size([18, 12, 10, 64]), torch.Size([18, 12, 10, 64])))


2023-09-24 11:56:48,470 [1792503362.py:73 in pre_forward] INFO - pre model.decoder.layers.2 forward,
 args:(torch.Size([18, 10, 768]),),
 kwargs: {'attention_mask': torch.Size([18, 1, 10, 10]), 'layer_head_mask': <class 'NoneType'>, 'past_key_value': <cla

In [40]:
model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 768, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 768)
      (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-11): 12 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), ep

In [33]:
args_type_set, kwargs_type_set, output_type_set

({int, torch.Tensor}, {NoneType, bool, torch.Tensor}, {torch.Tensor, tuple})

In [ ]:
layer_module._hf_hook

In [ ]:
# load / offload
#     module object, .dat file path
#     layer: pre / post forward hook

In [ ]:

# def compute_activation_assignment(num_layers, offload_config: Policy):
#     logging.debug(f"<compute_activation_assignment> enter")
#     gpu_batch_limit = int(offload_config.num_gpu_batches * offload_config.act_gpu_percent)
#     cpu_batch_limit = int(offload_config.num_gpu_batches * (offload_config.act_gpu_percent + offload_config.act_cpu_percent))
#     logging.debug(f"<compute_activation_assignment> gpu_batch_limit: {gpu_batch_limit}, cpu_batch_limit: {cpu_batch_limit}")
    
#     act_assign_dict = {}
#     for l in range(num_layers):
#         for i in range(offload_config.num_gpu_batches):
#             act_key = f"layer.{l}_index.{i}"
#             if i < gpu_batch_limit:
#                 device = 'cuda'
#             elif i < cpu_batch_limit:
#                 device = 'cpu'
#             else:
#                 device = 'disk'
#             act_assign_dict[act_key]= {'assigned_device': device}
#     return act_assign_dict


# def compute_kv_cache_assignment(num_layers, offload_config: OffloadConfig):
#     logging.debug(f"<compute_kv_cache_assignment> enter")
#     gpu_batch_limit = int(offload_config.num_gpu_batches * offload_config.cache_gpu_percent)
#     cpu_batch_limit = int(offload_config.num_gpu_batches * (offload_config.cache_gpu_percent + offload_config.cache_cpu_percent))
#     logging.debug(f"<compute_kv_cache_assignment> gpu_batch_limit: {gpu_batch_limit}, cpu_batch_limit: {cpu_batch_limit}")
    
#     act_assign_dict = {}
#     for l in range(num_layers):
#         for i in range(offload_config.num_gpu_batches):
#             key_cache_key = f"key_layer.{l}_index.{i}"
#             value_cache_key = f"key_layer.{l}_index.{i}"
#             if i < gpu_batch_limit:
#                 device = 'cuda'
#             elif i < cpu_batch_limit:
#                 device = 'cpu'
#             else:
#                 device = 'disk'
#             act_assign_dict[key_cache_key] = {'assigned_device': device}
#             act_assign_dict[value_cache_key] = {'assigned_device': device}
#     return act_assign_dict
